다음 데이터 세트를 이용하여 다이아몬드 가격예측(회귀) 

url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv" diamonds = pd.read_csv(url) 

1. ML (RF, DT, LR) 수행

2. Dense layer만 이용 FNCC 구현

3. 순환 데이터 변환후 CNN 구현



각 단계별로 수행완료후

검사받은 이후에 다음 단계 진행

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv"
df = pd.read_csv(url)
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [3]:
print(np.unique(df['cut'], return_counts=True))
print(np.unique(df['color'], return_counts=True))
print(np.unique(df['clarity'], return_counts=True))

(array(['Fair', 'Good', 'Ideal', 'Premium', 'Very Good'], dtype=object), array([ 1610,  4906, 21551, 13791, 12082]))
(array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype=object), array([ 6775,  9797,  9542, 11292,  8304,  5422,  2808]))
(array(['I1', 'IF', 'SI1', 'SI2', 'VS1', 'VS2', 'VVS1', 'VVS2'],
      dtype=object), array([  741,  1790, 13065,  9194,  8171, 12258,  3655,  5066]))


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

origin_label_cut = df['cut'].copy()
origin_label_color = df['color'].copy()
origin_label_clarity = df['clarity'].copy()

df['cut'] = label_encoder.fit_transform(df['cut'])
df['color'] = label_encoder.fit_transform(df['color'])
df['clarity'] = label_encoder.fit_transform(df['clarity'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  int64  
 2   color    53940 non-null  int64  
 3   clarity  53940 non-null  int64  
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 4.1 MB


In [6]:
df.describe()

,carat,cut,color,clarity,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,2.553003,2.594197,3.835150,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.027708,1.701105,1.724591,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,0.000000,0.000000,0.000000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,2.000000,1.000000,2.000000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,2.000000,3.000000,4.000000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,3.000000,4.000000,5.000000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,4.000000,6.000000,7.000000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [7]:
len(df)

53940

# 1. ML (RF, DT, LR) 수행

## DT

In [8]:
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,2,1,3,61.5,55.0,326,3.95,3.98,2.43
1,0.21,3,1,2,59.8,61.0,326,3.89,3.84,2.31
2,0.23,1,1,4,56.9,65.0,327,4.05,4.07,2.31
3,0.29,3,5,5,62.4,58.0,334,4.20,4.23,2.63
4,0.31,1,6,3,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,2,0,2,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,1,0,2,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,4,0,2,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,3,4,3,61.0,58.0,2757,6.15,6.12,3.74


In [9]:
X = df.drop('price', axis=1).to_numpy()
y = df['price'].to_numpy()

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.1,
                                                    shuffle=True,
                                                    random_state=42)
print(X_train.shape, y_train.shape)

(48546, 9) (48546,)


In [11]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(max_depth=5,
                           random_state=42)
dt.fit(X_train, y_train)
r_squared_train = dt.score(X_train, y_train)
r_squared_test = dt.score(X_test, y_test)

print(f"R-squared-train: {r_squared_train}")
print(f"R-squared-test : {r_squared_test}")

R-squared-train: 0.9277509025817178
R-squared-test : 0.9324689745233762


In [12]:
from sklearn.metrics import mean_squared_error

y_preds = dt.predict(X_test)
print(f"MSE: {mean_squared_error(y_preds, y_test)}")

MSE: 1090388.5166770718


In [13]:
print(dt.feature_importances_)
print("==========================")
for (name, importance) in zip(df.columns[:-1], dt.feature_importances_):
    print(f'{name:<15} {importance:.4f}')

[0.66042348 0.         0.01919801 0.04756256 0.         0.
 0.0009539  0.27186206 0.        ]
carat           0.6604
cut             0.0000
color           0.0192
clarity         0.0476
depth           0.0000
table           0.0000
price           0.0010
x               0.2719
y               0.0000


## RF

In [14]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth=3,
                           random_state=42)
rf.fit(X_train, y_train)
r_squared_train = rf.score(X_train, y_train)
r_squared_test = rf.score(X_test, y_test)

print(f"R-squared-train: {r_squared_train}")
print(f"R-squared-test : {r_squared_test}")

R-squared-train: 0.8845283197619171
R-squared-test : 0.8875107937912216


In [15]:
y_preds = rf.predict(X_test)
print(f"MSE: {mean_squared_error(y_preds, y_test)}")

MSE: 1816304.992179181


## LR

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=20, max_iter=3)
lr.fit(X_train_scaled, y_train)
lr.score(X_train_scaled, y_train)

# C의 기본값은 1, alpha와는 반대로 값이 커질수록 규제가 완화됨
# max_iter로 반복 수 지정

/opt/anaconda3/envs/gc_dl/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.00803361759980225